Internship Interview Project

Topic : Building a  ML model to analyse the customer sentiments from kindle book reviews.

For this task I have used tensorflow as my primary model building and testing module.

The dataset I have used here is the openly available data of amazon kindle book reviews. This dataset consists of the identifier of the book, the book review, the reviewer name, rating, review time and other information. Out of these the ones I will be focusing on is the review and the rating. The goal is to classify the customer sentiment into positive and negative based on the rating and then build and train a model to learn the customer sentiment from the review text itself.

In [35]:
#importing necessary libraries
import pandas as pd
import tensorflow as tf



from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SimpleRNN, GRU


In [5]:

#load the data 
data = pd.read_csv(r"C:\Users\gervi\Desktop\Rupal_proj\archive\all_kindle_review .csv")


,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


To classify the customer sentiment into positive and negative, I have use the book rating as a proxy. Here I have defined the sentiment as follows: a rating of 3 or above is labelled positive (1) and a lower rating yields a negative sentiment (0).


Later the dataset is split into training and test set where test set constitutes 20% of the full dataset. Training model needs to be large to ensure good optimization of the model parameters.

To build the model, a relationship between the review text and the customer sentiment needs to be established. For this I first assign indices or tokens to the most frequently used words in the review text of the training set. Here a limit of 10000 most frequent words was chosen by me. This means that the most freqeunt word will have the token '1', the next one '2' and so on. 

In [31]:
#defining the sentiment, `1` for positive and `0` for negative based on the customer rating (here 3 is chosen as a threshold)
data['sentiment'] = data['rating'].apply(lambda rating : +1 if rating >= 3 else 0)


#splitting the data into training and testing sets, here reviewText is the feature and sentiment is the target
X_train, X_test, y_train, y_test = train_test_split(data['reviewText'], data['sentiment'], test_size=0.2, random_state=2)


#defining the maximum number of words to be used in the model and the maximum length of the review to be considered
nwords_max = 10000
len_max = 500

#This is to assing index or tokens to each word in the review based on its frequency, here the 10000 most frequent words are collected
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=nwords_max)
tokenizer.fit_on_texts(X_train)







In the next part I convert the review text into a sequence of tokens that were assigned in the previous block of code and the length of all reviews is truncated or padded to 500 units. This step converts our review text into sequences of length 500, which is later used to establish a relation with the sentiment (0 or 1).

In [32]:
#Now the words in the reviews are converted to sequence of indices or tokens based on the previous tokenization step for the training and test samples
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


#to ensure that each review is of the same length, the reviews are padded with zeros to make them of the same length i.e in this case
X_train_seq_trunc = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=len_max)
X_test_seq_trunc = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=len_max)



The most important step is now to build a model based on the nature of relationship between our feature (review text) and target (sentiment). For this purpose I decide to use a simple sequential model with a few add-on layers. The activation funtion is chosen as sigmoid for the obvious binary nature of the target.

In [28]:
#Building the model

#Here we are using a simple sequential model upon which an embedding layer, LSTM layer(for sentences) and a dense layer is added
kindle_model = Sequential()
kindle_model.add(Embedding(nwords_max,128,input_length=len_max))
kindle_model.add(LSTM(64,dropout=0.2, recurrent_dropout=0.2))
kindle_model.add(Dense(1, activation='sigmoid'))

kindle_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

c:\Users\gervi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In this crucial step I fit the model with my training sample set in batches of size 64 over 5 epochs and later use the test set to validate the model. The accuracy of the model is also calculated at each epoch.

In [34]:
#fitting the model

kindle_model.fit(X_train_seq_trunc, y_train, batch_size=64, epochs=5, validation_data=(X_test_seq_trunc1, y_test1))


Epoch 1/5


150/150 ━━━━━━━━━━━━━━━━━━━━ 49s 324ms/step - accuracy: 0.7438 - loss: 0.5056 - val_accuracy: 0.8200 - val_loss: 0.3990
Epoch 2/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 50s 330ms/step - accuracy: 0.8645 - loss: 0.3250 - val_accuracy: 0.8462 - val_loss: 0.3691
Epoch 3/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 51s 338ms/step - accuracy: 0.9080 - loss: 0.2384 - val_accuracy: 0.8383 - val_loss: 0.3846
Epoch 4/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 51s 339ms/step - accuracy: 0.9292 - loss: 0.1933 - val_accuracy: 0.8308 - val_loss: 0.4250
Epoch 5/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 50s 335ms/step - accuracy: 0.9353 - loss: 0.1798 - val_accuracy: 0.8350 - val_loss: 0.4901
Epoch 1/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 50s 333ms/step - accuracy: 0.9512 - loss: 0.1325 - val_accuracy: 0.8221 - val_loss: 0.4737
Epoch 2/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 50s 333ms/step - accuracy: 0.9577 - loss: 0.1200 - val_accuracy: 0.8196 - val_loss: 0.5500
Epoch 3/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 49s 330ms/step - accuracy: 0.9322 - loss: 0.1861 - val_accura

In [17]:
#to evaluate the performance of the model on the test data set the loss and accuracy are calculated
score, accu = kindle_model.evaluate(X_test_seq_trunc, y_test, batch_size=64)
print('test accuracy: ', accu)

38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.8291 - loss: 0.4630
test accuracy:  0.8374999761581421


The accuracy of this model is now obtained which is around 0.84. This indicated that our simple model is fairly good at gauging the customer sentiment just from the text written in the review. Such a model would be very useful to make useful and practical product decisions for the business to make sure they are adhering to the feedback received from the customers in general. Specifically in this case of Amazon Kindle, such a ML model would help simply and understand customer sentiment for different genres of book and along with the demographic information of the customer can aid make informed business decisions. 

As for the model and its evaluation, a lot more can be build up on this project like using different models and playing around with the hyperparameters like epochs, maximum number of word etc. Also for the evaluation of the model a confusion matrix or a loss function curve can be computed as well to help visualize the fitting better. However for the sake of this project I have decided to keep it simple and concise. 